In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
using IndexedTables

┌ Info: Precompiling RollingFunctions [b0e4dd01-7b14-53d8-9b45-175a3e362653]
└ @ Base loading.jl:1317


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5404652031231561250\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9334577024153274595\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5404652031231561250\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [3]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [4]:
start_date = DateTime(2021, 5, 20)
end_date = DateTime(2021, 5, 27)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

10081

In [5]:
# xdf = DataFrame(ut = Int[], a = String[])
# push!(xdf, (ut = 1, a = "A"))
# push!(xdf, (ut = 2, a = "B"))
# r = xdf[xdf.a .== "A",:]
# r
# push!(xdf, (ut = 1, a = "A"))
# xdf

In [6]:
# xdfq = dfq[(dfq.src .== :close) .& (dfq.lvl .!= "HH"), :]
# xdfq.ut
# dfq[(dfq.src .== :ema100) .& (dfq.lvl .== ""), :]
len = nrow(dfq)
    sdfq = dfq[max(1,len-400):len,:]
    xdfq = sdfq[(sdfq.src .== :close) .& (sdfq.lvl .== ""), ["ut", "val"]]

LoadError: UndefVarError: dfq not defined

In [7]:
roll_sma(n,arma,i) = i == 1 ? ar[i] :
        i < n+1 ? 
            arma[i-1] + (ar[i] - last(arma)) / i :
            arma[i-1] + (ar[i] - ar[i-n])/n

roll_sma(n,arma,i) = i == 1 ? ar[i] :
        i < n+1 ? 
            arma[i-1] + (ar[i] - last(arma)) / i :
            ar[i] * (2/(n+1)) + arma[i-1] * (1 - (2/(n+1)))
#             arma[i-1] + (ar[i] - ar[i-n])/n




roll_sma (generic function with 1 method)

In [437]:
# ri(a,i) = a[length(a) - i + 1]
# rip(a,i) = ri(a,i)[2]
# rit(a,i) = ri(a,i)[1]

pushdfq(ut, src, lvl, val, nrec) = begin
#     println("$ut, $src, $lvl, $val, $nrec")
    push!(dfq, (ut=ut,src=src,lvl=lvl,val=val))
    if nrec == 0 return end    
    len = nrow(dfq)
    sdfq = dfq[max(1,len-1000):len,:]
    xdfq = sdfq[(sdfq.src .== src) .& (sdfq.lvl .== lvl), ["ut", "val"]]
    len = length(xdfq.ut)
    if len < 3 return end
#     println(arsrc[len, "val"])
    if xdfq.val[len] > xdfq.val[len-1] && xdfq.val[len-2] > xdfq.val[len-1] #low
#         push!(dfq, (ut=ut,src=src,lvl=lvl,val=val))
        pushdfq(xdfq.ut[len-1], src, "L$lvl", xdfq.val[len-1], nrec - 1)
    elseif xdfq.val[len] < xdfq.val[len-1] && xdfq.val[len-2] < xdfq.val[len-1] #high
#         push!(dfq, (ut=ut,src=src,xlvl=lvl,val=val))
        pushdfq(xdfq.ut[len-1], src, "H$lvl", xdfq.val[len-1], nrec - 1)
    end

end

pushdfq (generic function with 3 methods)

In [289]:
ar = cc
# ema = Indicators.ema
# ema26 = ema(ar,n=26)
# ema50 = ema(ar,n=330)
ema100 = ema(ar,n=100)
# ema200 = ema(ar,n=200)
# sma = Indicators.sma
# sma26 = sma(ar,n=26)
# sma50 = sma(ar,n=50)
sma100 = sma(ar,n=100)
# sma200 = sma(ar,n=200)
# sma400 = sma(ar,n=400)
# sma800 = sma(ar,n=800)
1

1

In [355]:
# push!(dfq, (ut=1,src=:close,lvl="lvl",val=4))

,ut,src,lvl,val
,Int64,Symbol,String,Float64
1,1621468800,close,,37143.1
2,1621468860,close,,36868.3
3,1621468920,close,,37156.5
4,1,close,lvl,4.0


In [438]:
dfq = DataFrame(ut = Int64[], 
        src = Symbol[],
        lvl = String[],
        val = Float64[]
    )
arsma100 = []
arema100 = []

for i in 1:length(ar)
    pushdfq(ut[i], :close, "", ar[i], 3)
    push!(arsma100, roll_sma(100,arsma100,i))
    push!(arema100, roll_sma(100,arema100,i))
    pushdfq(ut[i], :sma100, "", last(arsma100), 3)
    pushdfq(ut[i], :ema100, "", last(arema100), 3)
end

In [315]:

# arsma100 = Float64[]
# zema100 = dfq[(dfq.src .== :ema100),:]
# qema100 = zema100[zema100.lvl .== "", :]
# dfq

In [249]:
ar = cc

capital = 10000.
risk_percent = 0.01
qfee = 0.001
dfo = DataFrame(ut = Int64[], 
        order = Symbol[],
        reason = Symbol[],
        price = Float64[],
        stop = Float64[],
        qty = Float64[],
        pnl = Float64[],
        capital = Float64[]
    )

push!(dfo, (ut = ut[1], order = :long, reason = :toolong, price = 0, stop = 0, qty = 0, pnl = 0, capital = capital))

in_long_pos = false
qlbuysig = 0.98
qlsellsig = 1.015
qlstopsig = 0.99

ar = cc
# arsma100 = Float64[]
# arema100 = Float64[]

for i in 1:length(ar)
#     pushdfq(ut[i], :close, "", ar[i], 3)
#     push!(arsma100, roll_sma(100,arsma100,i))
#     push!(arema100, roll_sma(100,arema100,i))
#     pushdfq(ut[i], :sma100, "", last(arsma100), 3)
#     pushdfq(ut[i], :ema100, "", last(arema100), 3)
    
#     if i < 120 || length(qh) < 1 || length(ql) < 1  continue end
    
    ord = last(dfo)
    
    if ord.reason !== :enter # not in position
        if ar[i] <= qlbuysig * sma100[i]
            risk_size = capital * risk_percent
            stop_price = ar[i] * 0.99 * (1-qfee)
            
            buy_price = ar[i]*(1+qfee)
            buyq = risk_size / (buy_price - stop_price)
            
            push!(dfo, (ut = ut[i], order = :long, reason = :enter, price = buy_price, qty = buyq,
                    stop = stop_price, pnl = 0, capital = capital))
            
            in_long_pos = true
        end
    else
        if ar[i] >= ord.price * qlsellsig  # target         
            sell_price = ar[i]*(1-qfee)
            profit_loss = ord.qty * (sell_price - ord.price)
            capital = capital + profit_loss
            push!(dfo, (ut = ut[i], order = :long, reason = :target, price = sell_price,
                    stop = 0, qty = ord.qty, pnl = profit_loss, capital = capital))
            
            in_long_pos = false
#             println("TARGET:\t$profit_loss, ")
            
        elseif ar[i] <= ord.stop # SL
            sell_price = ar[i]*(1-qfee)
            profit_loss = ord.qty * (sell_price - ord.price)
            capital = capital + profit_loss
            
            push!(dfo, (ut = ut[i], order = :long, reason = :stoploss, price = sell_price, stop = 0, 
                    qty = ord.qty, pnl = profit_loss, capital = capital))
            in_long_pos = false
#             println("STOPLOSS:\t$profit_loss,")
        end
        
    end
#     println(capital)
    
end

In [91]:
# last(dfo).cap

In [439]:

xsteps(a) = vcat([[a[i-1][1],  a[i][1], a[i][1], NaN] for i in 2:length(a)]...)
ysteps(a) = vcat([[a[i-1][2],  a[i-1][2],  a[i][2], NaN] for i in 2:length(a)]...)
steps!(a;args...) = plot!(xsteps(a), ysteps(a);args...)

xqfind(ar,xut) = findall(x -> x[1] >= xut[1] && x[1] <= xut[length(xut)], ar)
xofind(ar,xut) = findall(x -> x.t >= xut[1] && x.t <= xut[length(xut)], ar)
ar = cc

ticks_per_step = 500
num_steps = Int(trunc(length(ar) / ticks_per_step * 4 ))

mp = @manipulate for 
            no_step in slider(1:num_steps; label="start", default=1)
#             utstep in slider(100:3000; label="zoom", default=800)
    
    sut = Int(trunc(no_step * ticks_per_step / 4))
    eut = min(sut + ticks_per_step, length(ar))

    xut = ut[sut:eut]
    xtt = tt[sut:eut]
    xar = ar[sut:eut]
    
    xema100= ema100[sut:eut]
    
    
#     xarema100= arema100[sut:eut]
#     xarsma100= arsma100[sut:eut]

    xdfq = dfq[(dfq.ut .>= first(xut)) .& (dfq.ut .<= last(xut)), :]
    xqema100 = xdfq[(xdfq.src .== :ema100) .& (xdfq.lvl .== ""), :]
    xqema100LL = xdfq[(xdfq.src .== :ema100) .& (xdfq.lvl .== "LL"), :]
    xqema100HH = xdfq[(xdfq.src .== :ema100) .& (xdfq.lvl .== "HH"), :]
    xHH = xdfq[(xdfq.src .== :close) .& (xdfq.lvl .== "HH"), :]
    xLL = xdfq[(xdfq.src .== :close) .& (xdfq.lvl .== "LL"), :]
    xHHH = xdfq[(xdfq.src .== :close) .& (xdfq.lvl .== "HHH"), :]
    xLLL = xdfq[(xdfq.src .== :close) .& (xdfq.lvl .== "LLL"), :]
    
    
#     c_LL = xdfq[(xdfq.src .== :close) .& (xdfq.lvl .== "LL") , ["ut", "val"]]

#     ftt = Dates.format.(xtt,"dd:HH:MM");
    
#     length(xtt) > 0 && latex(xtt[1])
    
    xticklabels = [ Dates.format(unix2datetime(x),"dd-HH:MM") for x in xut ]    
    f = plot(xut, xar, size=(1400,600), color=:blue, lw=0.5, legend = :outertopright, labels="close" )
    xtick_range = 1:30:length(xut)
    plot!(xticks=(xut[xtick_range],xticklabels[xtick_range]), xrot=45)

#     plot!(xut, xema100)
    plot!(xqema100.ut, xqema100.val)
    
#     scatter!(xLL.ut, xLL.val, markersize=4)
#     scatter!(xHH.ut, xHH.val, markersize=4)
#     scatter!(xHHH.ut, xHHH.val, markersize=4, label="xHHH")
#     scatter!(xLLL.ut, xLLL.val, markersize=4, label="xLLL")
    scatter!(xqema100LL.ut, xqema100LL.val, label="xqema100LL", markersize=4)
    scatter!(xqema100HH.ut, xqema100HH.val, label="xqema100HH", markersize=4)
    
#     xarema100
#     plot!(xut, xarsma100, label="arsma100", color=:green)
#     plot!(xut, xsma100, label="sma100", color=:red)
    
#     plot!(xut, xema100 .* 1.01, label="ema100", color=:red)
    plot!(xqema100.ut, xqema100.val, fillrange=xqema100.val * qlbuysig, fillalpha=0.1, label="xqema100 * $qlbuysig")
#     c_LL.ut
#     xqema100
#     nrow(c_LL) > 0 && scatter!(c_LL.ut, c_LL.val)
#     plot!(xut, xsma100, fillrange=xsma100*qlbuysig, fillalpha=0.1, label="xsma100 * $qlbuysig")
    
    
#     xql,xqh = ql[xqfind(ql,xut)],qh[xqfind(qh,xut)]
#     xqll,xqhl = qll[xqfind(qll,xut)],qhl[xqfind(qhl,xut)]
#     xqlh,xqhh = qlh[xqfind(qlh,xut)],qhh[xqfind(qhh,xut)]
    
#     xdfo = dfo[(dfo.ut .>= first(xut)) .& (dfo.ut .<= last(xut)), :]
#     xenters = xdfo[xdfo.or .== enter, :ut]
#     xstops = xdfo[xdfo.or .== enter, :ut]
#     xtargets = xdfo[xdfo.or .== target, :ut]
#     length(xenters) > 0 && vline!(xenters, linewidth=0.5, color=:blue)
#     length(xstops) > 0 && vline!(xstops, linewidth=0.5, color=:red)
#     length(xtargets) > 0 && vline!(xtargets, linewidth=0.5, color=:green)
   
#     scatter!(xqll, markersize=2, color=:red, label="xqll")
#     steps!(xqll, label="xqll")
#     scatter!(xqhl, markersize=2, color=:olive, label="xqhl")
#     steps!(xqhl, label="xqhl")
#      scatter!(xqhh, markersize=2, color=:red, label="xqhh")
#     steps!(xqhh, label="xqhh")
#     scatter!(xqhl, markersize=2, color=:olive, label="xqhl")
#     steps!(xqhl, label="xqhl")   
    
    # plot!(xsteps(qll), ysteps(qll), fillrange=ysteps(qll)*1.01, fillalpha=0.1)
    # steps!(qhl)



#     f
end
#     using Blink
#     w = Window()
#     body!(w, mp);
mp

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["start"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 80, :default => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
40, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000119307ec0, Task (runnable) @0x0000000119307ec0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",

In [ ]:
JSON.json([2,3])

In [ ]:
# import Pkg; Pkg.add("HTTP")
# using HTTP
HTTP.request("POST", "http://httpbin.org/post", [], "post body data")

In [ ]:
rootxchurl = "https://api.kucoin.com"
conf = (readtimeout = 10,
        pipeline_limit = 4,
        retry = false,
        redirect = false)
"$rootxchurl/api/v1/market/orderbook/level1?symbol=BTC-USDT"

In [ ]:
r = HTTP.get("$rootxchurl/api/v1/symbols";conf...)
1

In [ ]:
status, jsres = r.status, (String(r.body))
# r.json

In [ ]:
jsres

In [ ]:
res = JSON.parse(jsres)

In [ ]:
jsres

In [ ]:
res

In [ ]:
res["data"]